In [101]:
from pathlib import Path
from uuid import uuid4
from hashlib import md5
from tempfile import NamedTemporaryFile
import re
from shutil import copy

import shapely.geometry as geo
import viewscad
import vsketch
from solid import *
from solid.utils import *  # Not required, but the utils module is useful


In [45]:
def hash_string(string: str):
    return md5(string.encode()).hexdigest()[:7]

In [46]:
r = viewscad.Renderer()

In [104]:
def vsketch_to_scad_2d(sketch: vsketch.Vsketch, output_folder: Path = Path("output/")):
    with NamedTemporaryFile(suffix=".svg") as tempsvg:
        tempsvg = NamedTemporaryFile(suffix=".svg")
        sketch.save(tempsvg.name)
        svg_text = Path(tempsvg.name).read_text()
        # vsketch dynamically generates a timestamp
        # rather than parse the svg as XML, let's just regex it out
        svg_text = re.sub(r"^.*\<dc\:date\>.*\<\/dc\:date\>$", "", svg_text, flags=re.MULTILINE)
        svg_hash = hash_string(svg_text)
    
    svg_path = output_folder / (svg_hash + ".svg")
    sketch.save(svg_path)

    SCALE = 1 / 0.2645833333
    SCAD_TEMPLATE = f"""
    module svg_{svg_hash}()
    {{
        scale([{SCALE}, {SCALE}, 0])
        {{
            import("{svg_path.resolve()}");
        }};
    }}
    """.strip()

    scadfile_temp = output_folder / (svg_hash + ".scad")
    scadfile_temp.write_text(SCAD_TEMPLATE)
    scadfile = import_scad(scadfile_temp)
    shape_2d = getattr(scadfile, f"svg_{svg_hash}")()
    return shape_2d

def svg_to_scad_2d(svg: Union[str, Path], output_folder: Path = Path("output/")):
    if isinstance(svg, str):
        svg = Path(svg)
    with NamedTemporaryFile(suffix=".svg") as tempsvg:
        tempsvg = NamedTemporaryFile(suffix=".svg")
        svg_text = svg.read_text()
        # vsketch dynamically generates a timestamp
        # rather than parse the svg as XML, let's just regex it out
        svg_text = re.sub(r"^.*\<dc\:date\>.*\<\/dc\:date\>$", "", svg_text, flags=re.MULTILINE)
        svg_hash = hash_string(svg_text)
    
    svg_path = output_folder / (svg_hash + ".svg")
    copy(svg, svg_path)

    SCALE = 1 / 0.2645833333
    SCAD_TEMPLATE = f"""
    module svg_{svg_hash}()
    {{
        scale([{SCALE}, {SCALE}, 0])
        {{
            import("{svg_path.resolve()}");
        }};
    }}
    """.strip()

    scadfile_temp = output_folder / (svg_hash + ".scad")
    scadfile_temp.write_text(SCAD_TEMPLATE)
    scadfile = import_scad(scadfile_temp)
    shape_2d = getattr(scadfile, f"svg_{svg_hash}")()
    return shape_2d


FileNotFoundError: [Errno 2] No such file or directory: 'openscad'

In [75]:
vsk = vsketch.Vsketch()
vsk.size("220x220", landscape=False)

ls = geo.LineString(
    [
        (
            1,
            vsk.height / 2,
        ),
        (vsk.width - 1, vsk.height / 2),
    ]
)
vsk.geometry(ls.buffer(30))

shape_2d = vsketch_to_scad_2d(vsk)
o = linear_extrude(5)(shape_2d)
r.render(o)
scad_render_to_file(o, "filepath.scad")


Geometries in cache: 3
Geometry cache size in bytes: 21144
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.013
   Top level object is a 3D object:
   Facets:        260


'/Users/peter/projects/py-open-scad/vsketch_experimental/filepath.scad'

In [141]:
import numpy as np

diameter = 100
radius = diameter / 2
height = 5  # sphere diameter
sphere_segments = 25
lip_diameter = diameter - 5
lip_height = 3
design_extrusion_height = height + lip_height


def unit_xy(angle):
    return np.cos(angle), np.sin(angle)


n_spheres = 100
spheres = []
for angle in np.linspace(0, 2 * np.pi, n_spheres, endpoint=False):
    x, y = unit_xy(angle)
    x_c = x * radius
    y_c = y * radius
    s = translate([x_c, y_c, 0])(sphere(d=height, segments=sphere_segments))
    spheres.append(s)

sphere_ring = translate([0, 0, height / 2])(hull()(*spheres))
bottom_ring = translate([0, 0, height / 4])(
    cylinder(d=diameter + height, h=height / 2, center=True, segments=n_spheres)
)

lip = translate([0, 0, height])(
    rotate_extrude(angle=360, segments=n_spheres)(
        translate([(lip_diameter / 2) - (lip_height / 2), 0])(
            circle(d=lip_height, segments=sphere_segments)
        )
    )
)

inset = translate([0, 0, height / 2])(
    linear_extrude(height / 2)(
        translate([-110, -110])(
            svg_to_scad_2d(
                "/Users/peter/projects/sketches/3dprinter/output/flow-field-coaster-2022-08-13 13-45-11.svg"
            )
        )
    )
)

result = bottom_ring + sphere_ring + lip - inset
# r.render(result)
scad_render_to_file(result, "coaster.scad")


'/Users/peter/projects/py-open-scad/vsketch_experimental/coaster.scad'